## 라이브러리 불러오기

In [1]:
# API 호출
# ## https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057440
from urllib.request import Request, urlopen
# Request https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.Request
# urlopen https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.urlopen
from urllib.parse import urlencode, quote_plus
# urlenocde https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.urlencode
# quote_plus https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.quote_plus
from urllib.parse import unquote
import requests

# 시간 생성
import time
import datetime
from pytz import timezone, utc

# 폴더 자동 생성
import os

# 코드 스케줄링
import threading
# import schedule

# XML 파싱
import xml.etree.ElementTree as ET

# 입력데이터 데이터프레임 작성
import pandas as pd
import numpy as np

# DataFrame 화면 출력                                                # DataFrame 화면 출력 설정이 변경되어 있습니다 #
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None # default = 60


In [2]:
# 폴더 자동 생성 함수
# https://data-make.tistory.com/170
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

## 기본값 설정하기

In [3]:
routeId = 204000046
year = 2020
month = 11
day = 28

## Key Unquote

In [4]:
KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D' # 종현
apiKey = unquote(KEY)

## API 호출

In [5]:
# 노선정보항목조회 busrouteservice/info
# routeId의 정류장 목록을 받아옵니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/station'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
request = Request(url + queryParams)
request.get_method = lambda: 'GET'

# API를 호출하여 XML 형식으로 된 string 데이터를 변수 "oneLineXML" 에 저장합니다.
oneLineXML = urlopen(request).read().decode('utf8')
# oneLineXML

## XML 파싱

In [16]:
# xtree는 "openapi" 도메인 API 호출에서 "headerCd"에 상관없이 3개의 태그를 갖습니다.
# [comMsgHeader, msgHeader, msgBody]
xtree = ET.fromstring(oneLineXML)
# for branch in xtree:
#     print(branch)
msgBody = xtree[2]

tagList = ["stationSeq", "stationId", "stationName"]
stationList = []
for branch in msgBody:
    stationList.append([branch.find(tag).text for tag in tagList])

# station_df의 index가 0부터 시작하는 문제를 해결 하기 위해 rangeIndex를 지정하였습니다.
stationCount = len(stationList) # 뒤에서 다시 사용되므로 함수로 선언시 주의
rangeIndex = pd.RangeIndex(start=1, stop=stationCount+1)

station_df = pd.DataFrame(stationList, columns=tagList, index=rangeIndex)
station_df = station_df.astype({
    "stationSeq" : "int32",
    "stationId" : "int32", 
    "stationName" : "string"
    
})
# station_df

## 데이터 불러오기

In [7]:
subCategoryDict = {'dtt' : '/daily-time-table/', 
                   'dtt-log' : '/log/daily-time-table/', 
                   'rid' : '/route-id-list/',
                   'main-log' : '/log/main/',
                   'info' : '/BusInfo/',
                   'dtt-output' : '/out/daily-time-table/'
                  }

In [22]:
root = os.getcwd()
subCategory = subCategoryDict['dtt-output']
routeId = 204000046
folderPath = root + '/AWS/DATA' + subCategory + str(routeId)
createFolder(folderPath)
outputPath = folderPath + '/2020-11-28.txt'

In [23]:
root = os.getcwd()
subCategory = subCategoryDict['dtt']
routeId = 204000046
folderPath = root + '/AWS/DATA' + subCategory + str(routeId)
inputPath = folderPath + '/2020-11-28.txt'

In [25]:
inputPath

'/home/ubuntu/Data-Structure-Project/AWS/DATA/daily-time-table/204000046/2020-11-28.txt'

In [28]:
itemTagList = ['endBus', 'lowPlate', 'plateNo', 'plateType', 'remainSeatCnt', 'routeId', 'stationId', 'stationSeq', 'Nan']
# itemTagList = ['endBus', 'plateNo', 'plateType', 'remainSeatCnt', 'routeId', 'stationId', 'stationSeq', 'Nan']
itemTagList.insert(0, 'DateTime')
_df = pd.read_csv(inputPath, sep=' ', skiprows=[0], names=itemTagList)
_df = _df.drop(["Nan"], axis=1)
_df = _df.sort_values(['plateNo'])
_df = _df.dropna()
# _df = _df.drop(_df[_df["endBus"]=="API"].index)

# string을 datetime객체로 변환하기
_df['DateTime'] = pd.to_datetime(_df['DateTime'])
# dateTimeObj = datetime.datetime.strptime(_df['DateTime'], '%Y-%m-%dT%H:%M:%S.%f%z')
# _df['DateTime'] = _df.DateTime.str.split('.').str[0]
# _df['DateTime'] = _df.DateTime.str.replace('T', ' ')

_df = _df.astype({
    'endBus': 'int32', 
    'lowPlate': 'int32', 
    'plateNo': 'string', 
    'plateType': 'int32',
    'remainSeatCnt': 'int32', 
    'routeId': 'int32', 
    'stationId': 'int32', 
    'stationSeq': 'int32'
})
# _df

In [ ]:
# stationSeq 가 뒤바뀌는 사례도 발생합니다. 
# cf) routeId = 204000046, DateTime = 20-11-17, plateNo = 경기78아1147, index = 166
# _df[_df['plateNo']=='경기78아1178'].sort_values(by='DateTime').reset_index(drop=True).iloc[-60:] 

In [29]:
# 차량 운행기록을 "plateNo"를 기준으로 구분합니다.
plateNoList = _df.groupby(['plateNo']).size().index
plateNoList

Index(['경기70아7482', '경기70아8601', '경기70아8789', '경기70아8880', '경기77바1198',
       '경기78아1117', '경기78아1147', '경기78아1178'],
      dtype='object', name='plateNo')

In [ ]:
# 차량 번호 한개에 대해서 검사를 실행합니다.
temp_df =_df[_df['plateNo']=='경기78아1178'].sort_values(by='DateTime').reset_index(drop=True)
tempIndexList = []
for i in range(1, len(temp_df)):
    if(temp_df.iloc[i-1]['stationSeq'] > temp_df.iloc[i]['stationSeq']):
            tempIndexList.append(i)
            # print((temp_df.iloc[i-1]['stationSeq'] , temp_df.iloc[i]['stationSeq']))
            # print(i)
            
    tempIndexList.append(len(temp_df))
# display(temp_df.iloc[-60:].groupby('stationSeq').min())

In [30]:
# 정류장의 수 만큼 행을 갖는 빈 데이터프레임을 생성합니다. 변수 rangeIndex는 [XML파싱]에서 선언되어 있습니다.
rangeIndex = pd.RangeIndex(start=1, stop=stationCount+1)
_oneDay_df = pd.DataFrame(index = rangeIndex)

In [31]:
# 각 차량 번호를 기준으로 반복문을 수행합니다.
for plateNo in plateNoList:
    
    # 특정 차량번호와 같은 운행기록을 "DateTime을 기준으로 오름차순으로 정렬합니다."
    sameBusList = _df[_df['plateNo'] == plateNo].sort_values(['DateTime']).reset_index(drop=True)
#     display(sameBusList)
    
    
    # 버스는 뒤로가지 않으므로
    # "stationSeq"가 작아지는 인덱스 i의 리스트를 만듭니다.
    # API 호출 오류로 stationSeq가 작아지는 경우가 있습니다. 이 문제를 해결하기 위해 보정계수 diffK를 둡니다. 
    indexList = []
    diffK = 10 if ( True ) else 0
    for i in range(1,len(sameBusList)):
        if(sameBusList.iloc[i-1]['stationSeq'] > sameBusList.iloc[i]['stationSeq'] + diffK ):
            indexList.append(i)
#             print((sameBusList.iloc[i-1]['stationSeq'] , sameBusList.iloc[i]['stationSeq']))
            
    indexList.append(len(sameBusList))
    # print(indexList)
    
    # 첫번재 인덱스, 인덱스의 리스트, 마지막 인덱스를 조합하여 호차별로 운행기록을 구분합니다.
    index = 0
    tupleList = []
    for i in indexList:
        tupleList.append((index, i))
        small_df = sameBusList.iloc[index:i].groupby(by=['stationSeq']).min()
        # display(small_df)
        
        # 정류소 데이터프레임에 합칩니다.
        merge_df = pd.merge(station_df, small_df, how='outer',  left_index=True, right_index=True)        ###### ERROR 발생 히스토리 지점 by='stationSeq' 로 했을때 index가 0부터 시작
        # merge_df = merge_df.set_index(keys='stationSeq')
        
        if(0):
            # 1번 정류장에 대한 DateTime의 결측치가 많으면 1일 시간표에서 column 전체가 정렬이 안되므로 bfill방식으로 두번 결측치를 채웁니다.
            merge_df['DateTime'] = merge_df['DateTime'].interpolate(method='bfill', limit=1)
#             merge_df['DateTime'] = merge_df['DateTime'].interpolate(method='ffill', limit=1)
#             merge_df['DateTime'] = merge_df['DateTime'].interpolate(method='bfill', limit=1)


        if(0):
            # 결측치를 채울 경우 문제가 되는 데이터가 잘 드러나지 않으므로 결측치는 가장 마지막에 채우세요.
            # if의 인수가 1이면 결측치를 처리합니다. 결측치 처리를 하지 않으려면 if의 인수를 0으로 수정하세요. 
            # df.interpolate(method='linear' or 'polynomial')
            # 시간에 대한 결측치 처리
            merge_df['DateTime'] = merge_df['DateTime'].interpolate(method='backfill', limit=3)
            merge_df['DateTime'] = merge_df['DateTime'].interpolate(method='pad', limit=3)

            # 빈좌석에 대한 결측치 처리, limit 값이 다름에 주의
            merge_df['remainSeatCnt'] = merge_df['remainSeatCnt'].interpolate(method='linear', limit=2, limit_direction='both')

            merge_df['endBus'] = merge_df['endBus'].interpolate(method='pad', limit=3)        
            merge_df['endBus'] = merge_df['endBus'].interpolate(method='backfill', limit=3)

            merge_df['lowPlate'] = merge_df['lowPlate'].interpolate(method='pad', limit=3)        
            merge_df['lowPlate'] = merge_df['lowPlate'].interpolate(method='backfill', limit=3)

            merge_df['plateType'] = merge_df['plateType'].interpolate(method='pad', limit=3)        
            merge_df['plateType'] = merge_df['plateType'].interpolate(method='backfill', limit=3)

            merge_df['routeId'] = merge_df['routeId'].interpolate(method='pad', limit=3)        
            merge_df['routeId'] = merge_df['routeId'].interpolate(method='backfill', limit=3)

        _oneDay_df[str(plateNo)+'_'+str(index)] = merge_df['DateTime']
        # display(merge_df)
        index=i
#         print(str(plateNo)+'_'+str(index))
#         display(merge_df)
#         break;
#     break;

# _oneDay_df

In [32]:
def get_full_data_index(_oneDay_df):
    '''
    함수 [get_full_data_index(df)]은 데이터프레임 안에서 결측치가 하나도 없는 행의 index를 반환합니다.
    시간표를 배차 시간순으로 정렬하기 위해 데이터프레임의 첫번째 행을 기준으로 열(axis=1)을 정렬합니다.
    하지만 데이터프레임의 첫번째 행을 기준으로 정렬을 하게되면 결측치로 인하여 정렬이 안되는 열이 발생합니다.
    결측치가 없는 행을 찾기 위해 제작한 함수 입니다.

    '''
    for i in range(len(_oneDay_df.columns)):
        for j in range(len(_oneDay_df)):
            if(_oneDay_df.iloc[j].isnull().sum() == i):
                return j
            
        print("모든 행에 결측치가 존재합니다.")

In [33]:
# 출차시간(1행)을 기준으로 column을 정렬합니다. X > 결측치가 없는 행중 인덱스가 가장 작은 행
# oneDay_df = _oneDay_df.sort_values(by=1, axis=1)
lineIndex = get_full_data_index(_oneDay_df)+1
oneDay_df = _oneDay_df.sort_values(by=lineIndex, axis=1)


# datetime 객체를 string 객체로 변환합니다. 시간을 가지고 계산해야 하는 일은 이전에 실행하세요.
oneDayString_df = oneDay_df.astype('string')
for column in oneDayString_df.columns:
    oneDayString_df[column] = oneDayString_df[column].str.slice(start=10, stop=16)

# [column명 : 차량번호] station_df 와 oneDay_df를 합칩니다. datetime이 아닌 다른 타입의 오브젝트가 행에 붙게 되므로 지금부터는 편집이 어렵습니다.
_finalOneDay_df = pd.concat([station_df,oneDayString_df], axis=1)
_finalOneDay_df.set_index('stationSeq', drop=True, inplace=True)
# _finalOneDay_df

In [35]:
# 정렬된 column에 출차 순서를 배정합니다. (1호차, 2호차, ...)
columnCount = len(oneDayString_df.columns)
oneDayString_df.columns = np.arange(1,columnCount+1)

# [column명 : n호차] station_df 와 oneDay_df를 합칩니다. datetime이 아닌 다른 타입의 오브젝트가 행에 붙게 되므로 지금부터는 편집이 어렵습니다.
finalOneDay_df = pd.concat([station_df,oneDayString_df], axis=1)
df = finalOneDay_df.set_index('stationSeq', drop=True)
df

,stationId,stationName,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
stationSeq,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,206000010,운중동먹거리촌,<NA>,<NA>,05:40,<NA>,<NA>,<NA>,07:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,206000009,운중동푸르지오하임,<NA>,05:21,05:41,06:01,<NA>,<NA>,<NA>,07:21,<NA>,08:01,<NA>,<NA>,<NA>,09:21,09:41,10:01,<NA>,10:42,<NA>,<NA>,<NA>,<NA>,<NA>,13:06,13:31,13:56,<NA>,14:46,15:12,15:36,<NA>,16:26,<NA>,17:16,17:42,<NA>,<NA>,<NA>,19:20,<NA>,<NA>,<NA>,<NA>,21:26,<NA>,<NA>,22:40,23:12
3,206000658,한빛교회.월든힐스아파트,<NA>,05:22,05:42,06:02,06:22,06:41,07:01,<NA>,07:42,08:02,08:21,<NA>,<NA>,<NA>,<NA>,10:02,10:21,<NA>,<NA>,<NA>,11:51,12:16,12:41,13:07,<NA>,13:57,<NA>,14:47,15:13,15:37,16:02,<NA>,<NA>,<NA>,<NA>,18:07,18:31,18:56,19:21,<NA>,<NA>,20:36,<NA>,21:27,21:51,22:16,22:41,<NA>
4,206000558,운중초등학교,<NA>,<NA>,<NA>,06:03,06:23,06:42,07:02,07:22,07:43,<NA>,<NA>,<NA>,09:02,09:22,09:42,10:03,10:23,10:43,11:02,11:27,11:53,12:17,12:43,13:08,<NA>,<NA>,<NA>,14:48,15:15,15:38,<NA>,16:27,16:52,17:17,17:43,18:08,<NA>,18:57,<NA>,<NA>,<NA>,20:38,21:05,21:28,21:52,22:17,22:42,23:13
5,206000007,운중중학교,<NA>,05:23,05:43,06:04,06:24,<NA>,<NA>,07:23,07:44,08:03,08:23,<NA>,09:03,<NA>,09:43,10:04,10:24,10:44,11:03,11:29,<NA>,<NA>,12:44,13:09,13:33,13:58,<NA>,14:49,15:16,15:39,16:04,<NA>,16:53,17:19,17:44,18:09,18:33,18:58,19:23,<NA>,<NA>,20:39,21:06,21:29,21:53,22:18,22:43,23:14
6,206000006,운중동행정복지센터,<NA>,05:24,05:44,06:05,<NA>,06:44,07:04,07:24,07:45,08:04,08:25,08:44,<NA>,09:24,09:45,10:05,10:25,10:45,11:05,11:30,11:54,12:19,12:45,13:10,13:34,14:00,<NA>,14:50,15:18,15:40,16:05,16:29,16:54,17:20,17:45,18:10,<NA>,18:59,19:25,19:50,<NA>,<NA>,21:07,21:30,<NA>,<NA>,<NA>,23:15
7,206000005,뫼루니육교,<NA>,05:25,05:46,06:07,06:25,06:46,07:06,07:25,<NA>,08:05,08:26,08:47,09:05,<NA>,09:47,10:07,10:27,10:47,<NA>,<NA>,11:55,12:23,12:46,13:11,<NA>,14:02,14:27,<NA>,15:20,<NA>,16:07,<NA>,16:56,17:22,17:48,18:12,18:35,19:01,19:26,<NA>,<NA>,20:40,21:08,21:31,21:55,22:20,22:45,23:16
8,206000004,판교원마을1.2단지.판교도서관,<NA>,05:27,05:47,06:08,<NA>,06:47,<NA>,<NA>,<NA>,08:08,08:28,08:48,09:07,09:28,09:48,10:08,10:28,10:48,<NA>,11:32,11:57,12:25,12:48,13:14,13:37,14:05,<NA>,14:53,15:23,15:43,16:10,16:32,16:57,17:23,17:51,18:13,<NA>,19:02,19:27,<NA>,<NA>,<NA>,21:10,21:32,<NA>,<NA>,22:47,<NA>
9,206000555,판교청소년수련관.판교종합사회복지관,<NA>,05:28,05:48,<NA>,06:25,<NA>,07:08,07:28,07:48,<NA>,<NA>,08:50,09:10,09:30,<NA>,<NA>,<NA>,<NA>,11:08,11:33,11:58,12:26,<NA>,13:15,<NA>,14:06,14:28,14:56,<NA>,15:45,16:12,16:34,17:00,<NA>,17:53,<NA>,18:38,<NA>,<NA>,19:53,20:18,20:43,21:11,<NA>,21:57,<NA>,22:48,23:18


## 일일 시간표 데이터 csv로 내보내기

In [36]:
df.to_csv(outputPath, encoding='utf-8-sig')

In [ ]:
rootPath = 'C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/DailyTimeTable/' + str(routeId)
createFolder(rootPath)
outputPath = rootPath + '/' + str(routeId) + '_' + str(year)[-2:] + '-' + str(month) + '-' + str(day) + '.csv'
df.to_csv(outputPath, encoding='ANSI')

# CSV 파일을 읽어서 EXCEL파일로 내보낸다 : 블로그 게시 목적

## 일일 시간표 데이터 csv 읽어오기

In [ ]:
df = pd.read_csv(outputPath, encoding='ANSI')

## 노선번호 받아오기

In [ ]:
KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D' # 종현
apiKey = unquote(KEY)

# 노선정보항목조회 busrouteservice/info
# 해당 노선에 대한 노선번호, 기점/종점 정류소, 첫차/막차시간, 배차간격, 운행업체 등의 운행계획 정보를 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
# print(url+queryParams)

request = Request(url + queryParams)
request.get_method = lambda: 'GET'
oneLineXML = urlopen(request).read().decode('utf8')
# print(oneLineXML)

xtree = ET.fromstring(oneLineXML)
resultCode = int(xtree[1].find("resultCode").text)
msgBody = xtree[2]

busRouteInfoItem = msgBody[0]
i=0
for info in busRouteInfoItem:
    print(f"{i} {info.tag} : {info.text}")
    i+=1

In [ ]:
routeName = busRouteInfoItem.find("routeName").text

## Excel로 내보내기

In [ ]:
rootPath = 'C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/ExcelExport/'
excelFileName = str(routeName) + '.xlsx'
filePath = rootPath + excelFileName

createFolder(rootPath)
print(filePath)

### 배포용 DataFrame 수정

In [ ]:
export_df = df.set_index('stationName').drop('stationId', axis=1)

### 내보내기

In [ ]:
#-- write an object to an Excel sheet using pd.DataFrame.to_excel()
export_df.to_excel(filePath, # directory and file name to write
            sheet_name = '평일', 
            na_rep = 'NaN', 
            float_format = "%.2f", 
            header = True, 
            #columns = ["group", "value_1", "value_2"], # if header is False
            index = True, 
            index_label = "정류장", 
            startrow = 0, 
            startcol = 0, 
            engine = 'openpyxl', 
            freeze_panes = (1, 1)
            )

## Excel 접근하여 수정하기
https://naakjii.tistory.com/56

In [ ]:
wb = openpyxl.load_workbook(filePath)
sheet = wb['평일']

In [ ]:
sheet.max_column, sheet.max_row

In [ ]:
sheet.column_dimensions['A'].width = 14

In [ ]:
for cell in sheet['A']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center')
#     cell.fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=openpyxl.styles.colors.Color(rgb='00FF00'))

In [ ]:
wb.save(filePath)

In [ ]:
wb.close()